# Tier 1 Facilities in High Quality Transit Areas
<hr style="border:2px solid #8CBCCB">

In [ ]:
%%capture
import warnings
warnings.filterwarnings('ignore')

import altair as alt
import branca
import folium
import geopandas as gpd
import intake
import pandas as pd

from IPython.display import Markdown, HTML

import utils
from shared_utils import geography_utils, map_utils, styleguide
from shared_utils import calitp_color_palette as cp
import B1_layers_to_plot
import B2_chart_utils as chart_utils

alt.themes.register("calitp_theme", styleguide.calitp_theme)
catalog = intake.open_catalog("./*.yml")

In [ ]:
df = catalog.tier1_facilities_processed.read()
facilities, hqta_corr = B1_layers_to_plot.layers_to_plot()

## Key Takeaways

In [ ]:
def aggregate_stats(df, group_cols):

    df2 = geography_utils.aggregate_by_geography(
        df, 
        group_cols = group_cols,
        sum_cols = ["sqft"],
        nunique_cols = ["sheet_uuid"]
    ).rename(columns = {"sheet_uuid": "num_facilities"})

    df2 = df2.assign(
        pct = df2.num_facilities.divide(df2.num_facilities.sum()).round(3)
    )
    
    return df2

In [ ]:
hqta_by_category = aggregate_stats(facilities, ["category"])
statewide_by_category = aggregate_stats(df, ["category"])

In [ ]:
def grab_caption_values(df):
    pct = round(df.pct.iloc[0] * 100)
    num = df.num_facilities.iloc[0]
    
    return pct, num

In [ ]:
# Grab values needed in caption
ca_facilities = statewide_by_category.num_facilities.sum()
hqta_facilities = hqta_by_category.num_facilities.sum()

ca_pct, ca_num = grab_caption_values(
    statewide_by_category[statewide_by_category.pct==statewide_by_category.pct.max()])

hqta_pct, hqta_num = grab_caption_values(
    hqta_by_category[hqta_by_category.pct==hqta_by_category.pct.max()])

ca_o_pct, ca_o_num = grab_caption_values(
    statewide_by_category[statewide_by_category.category=="office"])

hqta_o_pct, hqta_o_num = grab_caption_values(
    hqta_by_category[hqta_by_category.category=="office"])

In [ ]:
# Use for bar charts
by_district = aggregate_stats(facilities, ["district", "category"])
by_county = aggregate_stats(facilities, ["county_name", "category"])

In [ ]:
by_district2 = aggregate_stats(facilities, ["district"])
by_county2 = aggregate_stats(facilities, ["county_name"])

d4_pct, d4_num = grab_caption_values(by_district2[by_district2.district==4])
d7_pct, d7_num = grab_caption_values(by_district2[by_district2.district==7])

sac_pct, sac_num = grab_caption_values(by_county2[by_county2.county_name=="Sacramento"])
la_pct, la_num = grab_caption_values(by_county2[by_county2.county_name=="Los Angeles"])

In [ ]:
display(HTML("<h4>Facilities</h4>"))
display(Markdown(
    f"* Statewide, there are **{ca_facilities} Tier 1 facilities.**"
))

display(Markdown(
    f"* **{hqta_facilities} ({round(hqta_facilities/ca_facilities * 100, 1)}%) "
    "facilities fall within HQTAs.**"
))


display(HTML("<h4>Categories</h4>"))
display(Markdown(
    f"* Statewide and in HQTAs, **maintenance is the largest category**, "
    f"with {ca_num} locations statewide ({ca_pct}%), "
    f"and {hqta_num} locations in HQTAs ({hqta_pct}%). "
))

display(Markdown(
    f"* Statewide, **offices** account for only {ca_o_pct}% ({ca_o_num}) of locations, "
    f"yet **in HQTAs, make up {hqta_o_pct}% ({hqta_o_num})** of locations."
))


display(HTML("<h4>Districts</h4>"))
display(Markdown(
    "* **Most locations are located in Districts 4 and 7.**"
))

display(Markdown(
    f"* District 4: {d4_num} locations ({d4_pct}%)"
))

display(Markdown(
    f"* District 7: {d7_num} locations ({d7_pct}%)"
))

display(HTML("<h4>Counties</h4>"))
display(Markdown(
    "* **Most locations are located in Sacramento and Los Angeles Counties.**"
))

display(Markdown(
    f"* Sacramento County: {sac_num} locations ({sac_pct}%)"
))

display(Markdown(
    f"* Los Angeles County: {la_num} locations ({la_pct}%)"
))

In [ ]:
hqta_donut = (chart_utils.make_donut_chart(hqta_by_category, 
                                           y_col = "num_facilities", 
                                           color_col = "category")
              .properties(title="HQTA", 
                          width = styleguide.chart_width * 0.7)
             )

all_facilities_donut = (chart_utils.make_donut_chart(statewide_by_category, 
                                                     y_col = "num_facilities",
                                                     color_col = "category")
                        .properties(title="Statewide", 
                                    width = styleguide.chart_width * 0.7
                                   )
                       )

donuts = (
    alt.hconcat(
        all_facilities_donut, 
        hqta_donut
    ).resolve_scale(theta='independent')
    .properties(title="Facility Categories Statewide and in HQTAs")
)

donuts = (styleguide.apply_chart_config(donuts)
          .configure_view(strokeWidth=0)
         )
donuts

In [ ]:
district_bar = chart_utils.make_bar_chart(by_district, "district")
district_bar

In [ ]:
county_bar = chart_utils.make_bar_chart(by_county, "county_name")
county_bar

## Map of Locations to Retrofit

In [ ]:
hqta_popup = {
    "hqta_type": "HQTA Type",
    "agency_name_primary": "Primary Agency",
    "agency_name_secondary": "Secondary Agency",
}

# plot_col needs to be numeric. categorical throws error.
# try it with dtype category?
# But, just use 1 color, and then it won't matter what plot_col was selected
hqta_plot_col = "sqft"
color_hqta = branca.colormap.StepColormap(
    colors=[cp.CALITP_CATEGORY_BRIGHT_COLORS[4]]
)


facilities_popup = {
    "facility_name": "Name",
    "address_arcgis_clean": "Address" ,
    "category": "Facility Category",
    "facility_type": "Facility Type"
}

facilities_plot_col = "sqft"
color_facilities = branca.colormap.StepColormap(
    colors=["black"]
)

LAYERS_DICT = { 
    "HQTA": {
        "df": hqta_corr,
        "plot_col": hqta_plot_col,
        "popup_dict": hqta_popup,
        "tooltip_dict": hqta_popup,
        "colorscale": color_hqta,
        "style_function": lambda x: {
            "fillColor": color_hqta(x["properties"][hqta_plot_col])
                if x["properties"][hqta_plot_col] is not None
                else "gray",
                "color": "#FFFFFF",
                "fillOpacity": 0.2,
                "weight": 0.2,
        }
    },
    "Facilities": {
        "df": facilities,
        "plot_col": facilities_plot_col,
        "popup_dict": facilities_popup,
        "tooltip_dict": facilities_popup,
        "colorscale": color_facilities,
     # https://stackoverflow.com/questions/50954840/displaying-radius-in-meters-with-folium
     # Circle shows radius in pixels, CircleMarker shows radius in meters (or whatever CRS is set) 
        "marker": folium.Circle(radius=500, fill_color="black", 
                             fill_opacity=0.9, 
                             color="black", weight=2),
     "highlight_function": lambda x: {"fillOpacity": 0.8},
     "zoom_on_click": True
    },   
}

FIG_WIDTH = 800
FIG_HEIGHT = 1_000

In [ ]:
#display(Markdown("#### Tier 1 Facilities"))
#color_facilities

In [ ]:
#display(Markdown("#### High Quality Transit Areas"))
#color_hqta

In [ ]:
LEGEND_URL = (
    'https://raw.githubusercontent.com/cal-itp/data-analyses/'
    'more-facilities/facilities_services/'
    'legend_facilities_hqta.png'
)

LEGEND_URL2 = (
    'https://github.com/cal-itp/data-analyses/'
    'raw/more-facilities/facilities_services/'
    'legend_facilities_hqta.png'
)

![image.png](https://raw.githubusercontent.com/cal-itp/data-analyses/more-facilities/facilities_services/legend_facilities_hqta.png)

In [ ]:
fig = map_utils.make_folium_multiple_layers_map(
    LAYERS_DICT,
    FIG_WIDTH,
    FIG_HEIGHT,
    zoom=map_utils.REGION_CENTROIDS["CA"]["zoom"],
    centroid=map_utils.REGION_CENTROIDS["CA"]["centroid"],
    title="Tier 1 Facilities in HQTA",
    legend_dict=None,
)

fig

## Facility Stats by District

In [ ]:
#district_list = sorted(facilities.district.unique())
#district_list

In [ ]:
def district_stats(df, district):
    df2 = df[df.district==district]
    
    display_cols = [
        "facility_name", "category", "address_arcgis_clean", 
        "facility_type", "sqft"
    ]
    
    # If there's no facility name, and it's an office, replace it with the District number
    df2 = (df2.assign(
        facility_name = df2.apply(lambda x: f"District {district} Office"
                                  if x.facility_name is None 
                                  else x.facility_name, axis=1)
        )[display_cols]
        .rename(columns = {"address_arcgis_clean": "address"})
    )
    
    df2.columns = df2.columns.str.replace('_', ' ').str.title()
    
    # Style the table
    df_style = (df2.style.format(subset=['Sqft'], 
               **{'formatter': '{:,.0f}'}, na_rep='')
                .set_properties(subset=[
                    'Facility Name', 'Address', 'Facility Type'], 
                    **{'text-align': 'left'})
                .set_properties(subset=['Category', 'Sqft'], 
               **{'text-align': 'center'})
                .set_table_styles([dict(selector='th',
                                        props=[('text-align', 'center')]
                                       )]
                                 )
             .hide(axis="index")
            )
          
    return df_style


### HQ

In [ ]:
district = 59
table = district_stats(facilities, district)
display(HTML(table.to_html()))

### District 3 - Marysville

In [ ]:
district = 3
table = district_stats(facilities, district)
display(HTML(table.to_html()))

### District 4 - Oakland

In [ ]:
district = 4
table = district_stats(facilities, district)
display(HTML(table.to_html()))

### District 7 - Los Angeles

In [ ]:
district = 7
table = district_stats(facilities, district)
display(HTML(table.to_html()))

### District 8 - San Bernardino

In [ ]:
district = 8
table = district_stats(facilities, district)
display(HTML(table.to_html()))

### District 11 - San Diego

In [ ]:
district = 11
table = district_stats(facilities, district)
display(HTML(table.to_html()))

### District 12 - Irvine

In [ ]:
district = 12
table = district_stats(facilities, district)
display(HTML(table.to_html()))